# Create the Annotated Call Centre Training Dataset (CCTD)

# Imports and Paths

In [1]:
# import time
import itertools
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from transformers import pipeline

from datasets import concatenate_datasets, Dataset, DatasetDict, load_dataset

from pathlib import Path
import sys
cwd = Path.cwd()
common_code_path = cwd.parent.parent / "plp_common_code"

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))

# from HF_Dataset_Helper import (hf_display_labels, hf_get_labels, hf_dataset_dict_describe, hf_dataset_describe, hf_display_random_dataset_rows, hf_display_dataset_row_idx)

from HF_Dataset_Helper import (hf_dataset_dict_describe, hf_dataset_describe, hf_dataset_display_random_rows, hf_dataset_display_row,
hf_dataset_get_labels, hf_dataset_display_labels)

# 1.0 Banking77 Dataset
Dataset composed of online banking queries annotated with their corresponding intents.<br>

BANKING77 dataset provides a very fine-grained set of intents in a banking domain. It comprises 13,083 customer service queries labeled with 77 intents. It focuses on fine-grained single-domain intent detection.
- https://huggingface.co/datasets/PolyAI/banking77

In [2]:
ds = load_dataset("PolyAI/banking77", trust_remote_code=True)
hf_dataset_dict_describe(ds)

type                : <class 'datasets.dataset_dict.DatasetDict'>

shape              : {'train': (10003, 2), 'test': (3080, 2)}

keys               : dict_keys(['train', 'test'])

column_names        : {'train': ['text', 'label'], 'test': ['text', 'label']}

Labels
train set / "label" is type ClassLabel -->
['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'dire

In [3]:
ds_train = ds['train']
ds_test = ds['test']

In [4]:
hf_dataset_display_labels(ds_test)

0  --> activate_my_card
1  --> age_limit
2  --> apple_pay_or_google_pay
3  --> atm_support
4  --> automatic_top_up
5  --> balance_not_updated_after_bank_transfer
6  --> balance_not_updated_after_cheque_or_cash_deposit
7  --> beneficiary_not_allowed
8  --> cancel_transfer
9  --> card_about_to_expire
10 --> card_acceptance
11 --> card_arrival
12 --> card_delivery_estimate
13 --> card_linking
14 --> card_not_working
15 --> card_payment_fee_charged
16 --> card_payment_not_recognised
17 --> card_payment_wrong_exchange_rate
18 --> card_swallowed
19 --> cash_withdrawal_charge
20 --> cash_withdrawal_not_recognised
21 --> change_pin
22 --> compromised_card
23 --> contactless_not_working
24 --> country_support
25 --> declined_card_payment
26 --> declined_cash_withdrawal
27 --> declined_transfer
28 --> direct_debit_payment_not_recognised
29 --> disposable_card_limits
30 --> edit_personal_details
31 --> exchange_charge
32 --> exchange_rate
33 --> exchange_via_app
34 --> extra_charge_on_statement
3

In [5]:
hf_dataset_display_random_rows(3, ds['train'])

row 1:              
Text: I bought something with my card and I was charged a fee
Label idx: 15 --> card_payment_fee_charged

row 2:              
Text: Why isn't my payment showing?
Label idx: 53 --> reverted_card_payment?

row 3:              
Text: My beneficiary is unable to transact
Label idx: 7 --> beneficiary_not_allowed



In [6]:
hf_dataset_display_random_rows(3, ds['test'])

row 1:              
Text: What steps do I take to receive another passcode?
Label idx: 44 --> passcode_forgotten

row 2:              
Text: My disposable virtual card will not work
Label idx: 72 --> virtual_card_not_working

row 3:              
Text: When adding money, what are the currencies you take?
Label idx: 54 --> supported_cards_and_currencies



In [7]:
ds_combined = concatenate_datasets([ds_train, ds_test])
ds_combined_text = ds_combined['text']
len(ds_combined_text)

13083

In [8]:
print("'ds_combined_text' has text only, no labels")
print(f"type: {type(ds_combined_text)}")
print(f"total rows: {len(ds_combined_text):,}")
print("=" * 50, '\n')
for i in range(5):
    print(ds_combined_text[i])

'ds_combined_text' has text only, no labels
type: <class 'list'>
total rows: 13,083

I am still waiting on my card?
What can I do if my card still hasn't arrived after 2 weeks?
I have been waiting over a week. Is the card still coming?
Can I track my card while it is in the process of delivery?
How do I know if I will get my card, or if it is lost?


# 2.0 Minds14 Dataset
MINDS-14 is training and evaluation resource for intent detection task with spoken data. It covers 14 intents extracted from a commercial system in the e-banking domain, associated with spoken examples in 14 diverse language varieties.
- https://huggingface.co/datasets/PolyAI/minds14

In [9]:
ds = load_dataset("PolyAI/minds14", name="en-US", trust_remote_code=True)
type(ds)

datasets.dataset_dict.DatasetDict

In [10]:
hf_dataset_dict_describe(ds)

type                : <class 'datasets.dataset_dict.DatasetDict'>

shape              : {'train': (563, 6)}

keys               : dict_keys(['train'])

column_names        : {'train': ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id']}

Labels
train set / "intent_class" is type ClassLabel -->
['abroad', 'address', 'app_error', 'atm_limit', 'balance', 'business_loan', 'card_issues', 'cash_deposit', 'direct_debit', 'freeze', 'high_value_payment', 'joint_account', 'latest_transactions', 'pay_bill']

train set / "lang_id" is type ClassLabel -->
['cs-CZ', 'de-DE', 'en-AU', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT', 'ko-KR', 'nl-NL', 'pl-PL', 'pt-PT', 'ru-RU', 'zh-CN']



In [11]:
ds_train = ds['train']

In [12]:
hf_dataset_display_labels(ds_train, lbl_col="intent_class")

0  --> abroad
1  --> address
2  --> app_error
3  --> atm_limit
4  --> balance
5  --> business_loan
6  --> card_issues
7  --> cash_deposit
8  --> direct_debit
9  --> freeze
10 --> high_value_payment
11 --> joint_account
12 --> latest_transactions
13 --> pay_bill


In [13]:
hf_dataset_display_random_rows(3, ds['train'], lbl_col="intent_class", text_col="english_transcription")

row 1:              
Text: hi I am traveling in Italy right now and I'm just trying to figure out if my
Label idx: 0 --> abroad

row 2:              
Text: show me my account balance please
Label idx: 4 --> balance

row 3:              
Text: hi I was wondering if there's any way to increase my withdrawal limit what's the maximum amount of money that I can withdraw from the ATM
Label idx: 3 --> atm_limit



In [14]:
# combine bank77 and minds14
type(ds['train']['english_transcription']), len(ds['train']['english_transcription'])

(list, 563)

In [15]:
print(len(ds_combined_text))
ds_combined_text = list(itertools.chain(ds_combined_text, ds['train']['english_transcription']))
print(len(ds_combined_text))

13083
13646


# 3.0 Schema-Guided Dialog (SGD) DSTC8
- https://huggingface.co/datasets/google-research-datasets/schema_guided_dstc8

In [16]:
ds = load_dataset("schema_guided_dstc8", trust_remote_code=True)
print(type(ds)), print(ds.shape)

<class 'datasets.dataset_dict.DatasetDict'>
{'train': (16142, 3), 'validation': (2482, 3), 'test': (4201, 3)}


(None, None)

In [17]:
desired_domain = "Banks_1"

def is_desired_domain(services: list):
    for svc in services:
        if svc == desired_domain:
            return True
        else:
            return False

bank_data = ds.filter(lambda x: is_desired_domain(x['services']))
bank_data

DatasetDict({
    train: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 265
    })
    validation: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 0
    })
    test: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 0
    })
})

In [18]:
ds = bank_data['train']
ds

Dataset({
    features: ['dialogue_id', 'services', 'turns'],
    num_rows: 265
})

In [19]:
# let's take a look a one of the Banking multi-turn dialog from the SGD dataset
# Random selection

max_idx = len(ds)-1

speaker_type = {0: "user", 1: "agent"}

counter = 0
max_counter = 1
while counter < max_counter:
    idx = random.randint(0, max_idx)
    dialogue = ds[idx]
    print(f"Dialogue ID: {dialogue['dialogue_id']}")
    print(f"Services: {', '.join(dialogue['services'])}")
    print("\nConversation:")
    print("=" * 50)
    turns = dialogue["turns"]
    for s, u, f in zip(turns["speaker"], turns["utterance"], turns["frames"]):
        print(f"{speaker_type[s]}:\n{u}\n")
    
    counter += 1    

Dialogue ID: 32_00077
Services: Banks_1

Conversation:
user:
I would like to check my balance please.

agent:
Would that be the balance for you checking or savings account?

user:
I would like the balance for the savings account please.

agent:
Your savings account balance is $22,398.89.

user:
Okay, that sounds about right.

agent:
Did you need to make a transfer?

user:
Yes, can you please transfer 980 dollars?

agent:
Yes I can, who would you like me to transfer the money to?

user:
Can you put it in the savings account for Maria?

agent:
Okay, So you would like for me to transfer $980 from your savings account and to put it in the savings account of Maria, correct?

user:
Yes that is correct.

agent:
Okay, your transfer has been completed.

user:
Great! Can you give me my new balance please?

agent:
Yes, your savings account balance is now $21,418.89.

user:
Okay, that sounds right.

agent:
Would you be needing anything else today?

user:
No, that will be all. Thank you.

agent:
Yo

In [20]:
# take a look at a specific multi-turn dialog
for multi_turn in ds["turns"][0:1]:
    for s, u, _ in zip(multi_turn["speaker"], multi_turn["utterance"], multi_turn["frames"]):    
        print(f"{speaker_type[s]}:\n{u}\n")

user:
What's my balance?

agent:
In checking or savings?

user:
In checking.

agent:
Your checking account has $5,118.77.

user:
Ok, I want to transfer some money.

agent:
To whom?

user:
To Amir.

agent:
And what amount would you like to transfer?

user:
Send $1,630.

agent:
Please confirm: Transfer $1,630 from your checking account to Amir's checking account.

user:
Confirmed.

agent:
Your transfer is complete.

user:
Thanks, what's my balance?

agent:
Your checking account has $3,488.77.

user:
Ok, bye.

agent:
Have a nice day.



In [21]:
# now let's take out the text to add to ds_combined_text
ds_sgd_text = []
for multi_turn in ds["turns"]:
    for s, u, _ in zip(multi_turn["speaker"], multi_turn["utterance"], multi_turn["frames"]):   
        ds_sgd_text.append(u)

print(type(ds_sgd_text))
print(len(ds_sgd_text))
print(ds_sgd_text[0])
print(ds_sgd_text[1])
print(ds_sgd_text[2])

<class 'list'>
4278
What's my balance?
In checking or savings?
In checking.


In [22]:
print(len(ds_combined_text))
ds_combined_text = list(itertools.chain(ds_combined_text, ds_sgd_text))
print(len(ds_combined_text))
print(type(ds_combined_text))

13646
17924


# 4.0 Save to Disk
- 1 copy to parquet, another to CSV (because I need to manually annotate a seed dataset for active learning), 1 to Dataset (local)
- Filename is 01_Create_Unannotated_CCTD.abc
- creating synthetic data will be done in 02
- Final combined CCTD will be in 03

In [45]:
data_0 = {'text': ds_combined_text}
df = pd.DataFrame(data_0)
print(df.shape)
display(df.head())

(17924, 1)


text
0                     I am still waiting on my card?
1  What can I do if my card still hasn't arrived ...
2  I have been waiting over a week. Is the card s...
3  Can I track my card while it is in the process...
4  How do I know if I will get my card, or if it ...

In [46]:
df.to_parquet('01_Create_Unannotated_CCT.parquet', engine='pyarrow', index=False)

In [47]:
df.to_csv('01_Create_Unannotated_CCT.csv', index=False)

In [48]:
data_0 = [{"text": row} for row in ds_combined_text]
print(type(data_0))
i = 0
j = 3
for row in data_0:
    if i > 3:
        break
    for k, v in row.items():
        print(k, ": ", v)
    i += 1

combined_ds = Dataset.from_list(data_0)
combined_ds.save_to_disk('01_Create_Unannotated_CCT')

<class 'list'>
text :  I am still waiting on my card?
text :  What can I do if my card still hasn't arrived after 2 weeks?
text :  I have been waiting over a week. Is the card still coming?
text :  Can I track my card while it is in the process of delivery?


Saving the dataset (0/1 shards):   0%|          | 0/17924 [00:00<?, ? examples/s]

# 5.0 Check Polarity of the CCTD with a 3 level sentiment classifier
To check the sentiment class distribution of the combined dataset and determine if action is required to correct class imbalance.

In [25]:
pipe = pipeline("text-classification", model="j-hartmann/sentiment-roberta-large-english-3-classes")

total_length = len(ds_combined_text)
# total_length = 1000

batch_size = 1000
num_batches = int(total_length / batch_size)

freq_d = {'neg': 0, 'neu': 0, 'pos': 0, 'neg_idx': [], 'neu_idx': [], 'pos_idx': []}

start_index = 0
for i in tqdm(range(num_batches)):    
    # print(f"start: {start_index} \tend: {start_index+batch_size}")
    sentiment_results = pipe(ds_combined_text[start_index: start_index+batch_size])
    for idx, r in enumerate(sentiment_results):
        if r['label'] == 'positive':
            freq_d['pos'] += 1
            freq_d['pos_idx'].append(start_index + idx)
        elif r['label'] == 'neutral':
            freq_d['neu'] += 1
            freq_d['neu_idx'].append(start_index + idx)
        elif r['label'] == 'negative':
            freq_d['neg'] += 1
            freq_d['neg_idx'].append(start_index + idx)
        else:
            raise ValueError("Unknown Sentiment Label")    
    start_index += batch_size

Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|█████████████████████████████████████████████████████████████████████████████████████████| 179/179 [1:11:44<00:00, 24.05s/it]


In [26]:
num_neg = freq_d['neg']
num_neu = freq_d['neu']
num_pos = freq_d['pos']
total_sentiments = num_neg + num_neu + num_pos
print(f"Negative: {num_neg}, Neutral: {num_neu}, Positive: {num_pos}")
print(f"Total: {total_sentiments:,}")

num_neg_pct = num_neg/total_sentiments
num_neu_pct = num_neu/total_sentiments
num_pos_pct = num_pos/total_sentiments

print(f"\nDistribution:\n negative\t: {num_neg_pct:,.2f}\n neutral\t: {num_neu_pct:,.2f}\n positive\t: {num_pos_pct:,.2f}")
print(f"Total: {num_neg_pct + num_neu_pct + num_pos_pct:,.2f}")

Negative: 4899, Neutral: 12440, Positive: 561
Total: 17,900

Distribution:
 negative	: 0.27
 neutral	: 0.69
 positive	: 0.03
Total: 1.00


In [56]:
# Sample Check Negative Sentiment
start = 0  # inclusive
end = len(freq_d['neg_idx'])    # exclusive

if end - start <= 0:
    random_indexes = None
else:
    number_to_sample = 20
    random_indexes = sorted(np.random.choice(np.arange(start, end), size=number_to_sample, replace=False))
    
print(f"total\t: {len(freq_d['neg_idx'])}")
if random_indexes is not None:
    print(f"sampling: {len(random_indexes)}")
    for i in random_indexes:    
        j = freq_d['neg_idx'][i]
        print(f"negative - {i} : {ds_combined_text[j]}")

total	: 4899
sampling: 20
negative - 634 : Cancel my transfer, please.
negative - 922 : I tried doing an in country transfer earlier this week. I thought it would be there by now but I still don't see the money. Can you check the status on my transfer?
negative - 1610 : I don't have a password.
negative - 2328 : Why do you keep declining my payment?I tried several times already with this card and it is just not working.
negative - 2471 : I wish to remove my account.
negative - 3119 : The exchange rate for my withdrawal was wrong.
negative - 3285 : Hello, I trying to top up with my card today but it didn't work. When I tried last time it worked but today it isn't. Could you take a look at it for me?
negative - 3358 : I transferred some funds but I don't see it
negative - 3376 : My transfer to my account is missing.
negative - 3427 : I see non-received cash in my app.
negative - 3687 : Why was the exchange rate incorrect for an item I recently bought?
negative - 3898 : I used my card for

In [55]:
# Sample Check Positive Sentiment

start = 0  # inclusive
end = len(freq_d['pos_idx'])    # exclusive

if end - start <= 0:
    random_indexes = None
else:
    number_to_sample = 20
    random_indexes = sorted(np.random.choice(np.arange(start, end), size=number_to_sample, replace=False))
    
print(f"total\t: {len(freq_d['pos_idx'])}")
if random_indexes is not None:
    print(f"sampling: {len(random_indexes)}")
    for i in random_indexes:
        j = freq_d['pos_idx'][i]
        print(f"positive - {i} : {ds_combined_text[j]}")

total	: 561
sampling: 20
positive - 34 : Have a good day.
positive - 49 : Have a good day.
positive - 52 : Okay cool thats nice
positive - 94 : Have a great day.
positive - 111 : Transfer successful.
positive - 129 : Enjoy your day.
positive - 140 : Have an awesome day.
positive - 152 : Great. Your transfer is complete.
positive - 200 : Okay, thank you so much and have a pleasant day.
positive - 215 : Have a nice day.
positive - 222 : See you soon!
positive - 283 : Oh, that's great.
positive - 300 : Enjoy your day!
positive - 362 : Fantastic. Can you please tell me the balance of my checking account?
positive - 377 : Have a nice day.
positive - 409 : I really like Drama.
positive - 435 : Yes, stockholm is perfect
positive - 441 : I'd really like a Thriller.
positive - 481 : Have a great day.
positive - 504 : An action movie is good.


In [57]:
# Sample Check Neutral Sentiment

start = 0  # inclusive
end = len(freq_d['neu_idx'])    # exclusive

if end - start <= 0:
    random_indexes = None
else:
    number_to_sample = 20
    random_indexes = sorted(np.random.choice(np.arange(start, end), size=number_to_sample, replace=False))
    
print(f"total\t: {len(freq_d['neu_idx'])}")
if random_indexes is not None:
    print(f"sampling: {len(random_indexes)}")
    for i in random_indexes:    
        j = freq_d['neu_idx'][i]
        print(f"neutral - {i} : {ds_combined_text[j]}")

total	: 12440
sampling: 20
neutral - 383 : What is this extra pound charge for?
neutral - 956 : How do I fix my card if it is broken?
neutral - 1545 : How do I cancel a transfer?
neutral - 1978 : I want to top up by using a credit card, which ones do you support?
neutral - 2446 : How do I modify my account details?
neutral - 2469 : why are all my identity details required
neutral - 2511 : I still want to use my account but haven't verified is that OK?
neutral - 3053 : i dont recognise some of these transactions, i think someone has my card details
neutral - 3421 : I lost my phone, what should I do?
neutral - 3922 : I want to know the source of my funds.
neutral - 4608 : I want to transfer some money from my other bank account into this one.
neutral - 5143 : I want a disposable virtual card, how do I do that?
neutral - 6378 : What is the tracking number for my card that was mailed?
neutral - 7391 : what is this charge on my account
neutral - 8501 : hey I need to pay a bill my electricit